# Find all must support dependent profiles in StructureDefinitions

- for hard coded path search all SD (in YAML) for all reference with must support = true profiles
- list of other profile that need to be supported
- incorporate into narrative generator...
- todo think about canonical too and !uscdi requirements as well! and inherited references from base. (such as Media) use snapshot instead?
  - check if inherited reference if emopty ( e.g. Media )


In [65]:
from pathlib import Path
from json import load, dumps
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from pprint import pprint
yml_path = Path('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml')
snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')  # use this 
my_path = snapshot_path
my_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/output')

In [78]:
mydict = {}
excludelist = []

whitelist = []

count = 0
cant_match_path = []

for enum,i in enumerate(my_path.glob('Struct*.json')):
    my_set = set()
    obj = y_load(i.read_text(),Loader=FullLoader) #dict
    if obj['id'] in excludelist or obj['type'] == 'Extension':
        continue
    # print(enum,i,)
    # print(obj['url'])
    #find references type elements in diff
    for element in obj['differential']['element']:
        # print(element['id'])
        # print()
    
        try:
            targetProfile = (type for type in element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
        except KeyError:
            # find snapshot_element
            try:
                snapshot_element =  (snapshot_element for snapshot_element in obj['snapshot']['element'] if snapshot_element['id'] == element['id']).__next__()
                # check if snapshot_element has a type reference
            except StopIteration:
                # print('id mismatch')
                pass
            else:
                try:
                    snapshot_reference = (type for type in snapshot_element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
                    # print(f"++++++++++++++++snapshot_reference={snapshot_reference}+++++++++++++++++++++")
                    my_set.update(snapshot_reference) #add list to set
                except KeyError:
                    # print('not a type = reference')
                    pass
                except StopIteration:
                    # print('not a type = reference')
                    pass
        except StopIteration:
            # print('no reference')
            pass
        else:
            ms_ext = None
            try:
                ms_ext = (type for type in element['type'] if type['code'] == 'Reference').__next__()['_targetProfile']
            except KeyError:
                # print('no  _targetProfile')
                pass
            except StopIteration:
                # print('stopIteration')
                pass

            # print(targetProfile, ms_ext)
            
            # print(element['id'])
            # print(dict(zip(targetProfile,ms_ext)))
            # print()
            try:
                zipped = dict(zip(targetProfile,ms_ext))
                ms_targetProfile = [k for k,v in zipped.items() if v["extension"][0]['valueBoolean']]
                # print(f"{element['id']} = {ms_targetProfile}\n")
                my_set.update(ms_targetProfile) #add list to set
            except TypeError:
                # print(f"{element['id']} = {targetProfile}\n")
                my_set.update(targetProfile) #add list to set
            
    # print(f'{my_set}\n')
    mydict[obj['url']] = list(my_set)
pprint(mydict)




{'http://hl7.org/fhir/us/core/StructureDefinition/head-occipital-frontal-circumference-percentile': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient'],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-blood-pressure': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-bmi': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-height': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-temperature': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-body-weight': [],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient'],
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam':